# Mobile App for Lottery Addiction

Playing in lotteries is an activity that many pursue initially for fun. However, for a significant segment of these people this form of gambling turns into a habit that can eventually escalate into addiction. Like other forms of compulsive gambling, lottery addicts risk spending from savings and are critically suceptible to predatory loans that can lead to accumulating debts. Most lottery addicts suffer from difficulty estimating the true odds of winning (which is often intentionally confounded by lottery designs) as well as the classic "gambler's fallacy" which leads to the notion that "a gamblers luck can turn around".

In this project, we'll imagine that a medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, so our role is to create the logical core of the app and calculate probabilities. 

For our first efforts we're going to focus on accurately capturing the behavior of the national [Canadian Lotto 6/49](https://en.wikipedia.org/wiki/Lotto_6/49). Lotto 6/49 is a lottery in which a total of six numbers ranging from 1 to 49 are drawn from a pool of forty-nine. If a ticket matches all six numbers exactly, a jackpot prize of at least $5,000,000 CAD is won. To train our model, we'll use a historical dataset of 3,665 drawings dating from 1982 to 2018 [hosted by Kaggle](https://www.kaggle.com/datascienceai/lottery-dataset).

Throughout this project we will seek to answer the following questions:
- What is the probability of winning the big prize with a single ticket?
- What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
- What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?


In [2]:
# Basic combinatorial functions
def factorial(n: int) -> int:
    """
    Returns the value of n! = n*(n-1)*(n-2)*...*1
    Inputs:
        n | int | value used to compute n!
    Returns:
        n! | int 
    """
    fac = 1
    for k in range(n):
        fac *= (k+1)
    return fac

def combinations(n: int, k: int) -> int:
    """
    Returns the number of combinations from a set of n items chosen k at a time.
    Inputs:
        n | int | number of items in set to pull from
        k | int | number of items chosen for each combination
    Returns
        C(n,k) | int | number of combinations of k items chosen from a set of n items.
    """
    assert k <= n, "n must be greater than or equal to k."
    return factorial(n)/(factorial(n-k)*factorial(k))